In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import numpy as np
from IPython.display import Image
import qutip
from qutip import *
from tqdm import tqdm
from qutip.solver import correlation_2op_1t
from matplotlib import rc
from scipy.optimize import curve_fit
from scipy.linalg import expm
from scipy.fft import ifftshift
import scipy.signal as signal
from scipy.special import factorial
import matplotlib as mpl
from matplotlib import cm
from brokenaxes import brokenaxes  # Librería para ejes rotos
import pandas as pd
from joblib import Parallel, delayed
from concurrent.futures import ProcessPoolExecutor
from optomechanics import *

In [2]:
#SYSTEM'S PARAMETERS
hbar=1
kappa= 250e-3 
g0 = 0.005  
Omega= 0.02*kappa #0.005 eV = 0.02*kappa, or 60e-3 eV
wm=100e-3
wc = 2.5 
gamma_m= 1e-3
n_th= np.linspace(0, 0.04, 10) #0.02 for T=300k
wl = wc+wm
wlist=np.linspace(2, 3, 10000)
Delta= wc-wl
alpha= Omega/(kappa/2 + 1j*Delta) 
alpha2= alpha.conjugate()*alpha
g = g0*abs(alpha)

In [3]:
T= 373 #ºC + 273.15 #K
freq=wm*(2*np.pi)/(4.135667696e-15) #rad/s
nth=1/(-1+np.exp(1.054571817e-34*freq/(1.380649e-23*T)))
print(nth)#(f"{nth:.3e}")

0.04662784214204384


In [4]:
#Analytical results
data = []

for i in range(len(n_th)):
    Gamma_plus, Gamma_minus, Gamma_opt = Transition_rates(g0, alpha2, kappa, Delta, wm)
    nb = pop_phonons(gamma_m, Gamma_opt, n_th[i], Gamma_plus)
    S = [emission_spec(w, nb, Gamma_plus, Gamma_minus, Gamma_opt, gamma_m, wl, wm) for w in wlist]
    data.append({
        'g0': g0,
        'Δ': Delta,
        'Omega': Omega,
        'nth': n_th[i],
        'Gamma_plus': np.real(Gamma_plus),
        'Gamma_minus': np.real(Gamma_minus),
        'Gamma_opt': np.real(Gamma_opt),
        'Analytical S': np.real(S),
        'Analytical w': wlist,
        'C': g**2/(kappa*gamma_m),
        'g/k': g/kappa
    })

# Convert to DataFrame
df = pd.DataFrame(data)

In [5]:
df

,g0,Δ,Omega,nth,Gamma_plus,Gamma_minus,Gamma_opt,Analytical S,Analytical w,C,g/k
0,0.005,-0.1,0.005,0.000000,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2489859716328869e-08, 1.2497356983380966e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
1,0.005,-0.1,0.005,0.004444,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2553315230317411e-08, 1.2560849678318156e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
2,0.005,-0.1,0.005,0.008889,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2616770744305952e-08, 1.2624342373255348e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
3,0.005,-0.1,0.005,0.013333,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2680226258294491e-08, 1.2687835068192538e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
4,0.005,-0.1,0.005,0.017778,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2743681772283035e-08, 1.275132776312973e-08...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
5,0.005,-0.1,0.005,0.022222,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2807137286271574e-08, 1.2814820458066924e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
6,0.005,-0.1,0.005,0.026667,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2870592800260113e-08, 1.2878313153004114e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
7,0.005,-0.1,0.005,0.031111,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2934048314248653e-08, 1.2941805847941306e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
8,0.005,-0.1,0.005,0.035556,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.2997503828237194e-08, 1.3005298542878496e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625
9,0.005,-0.1,0.005,0.040000,3.902439e-07,1.096191e-07,-2.806248e-07,"[1.3060959342225734e-08, 1.3068791237815687e-0...","[2.0, 2.0001000100010002, 2.000200020002, 2.00...",0.000098,0.000625


In [6]:
#NUMERICAL SIMULATION
#Hamiltonian construction
Nc = 12  # Number of cavity states
Nm = 10 # Number of mech states

a = tensor(destroy(N=Nc), qeye(Nm))
b = tensor(qeye(Nc), destroy(N=Nm))
num_b = b.dag() * b
num_a = a.dag() * a
#Choose Hamiltonian
H = hbar*Delta*num_a+hbar*wm*num_b+1j*Omega*(a.dag()-a)-g0*num_a*(b.dag()+b)
Hlin = hbar*wm*num_b+hbar*Delta*num_a-hbar*abs(alpha)*g0*(a.dag()+a)*(b.dag()+b)
c_ops=[]
for i in range(len(n_th)):
    c_ops.append([np.sqrt(kappa)*a, np.sqrt((n_th[i]+1)*gamma_m)*b, np.sqrt(n_th[i]*gamma_m)*b.dag()])

In [7]:
%%time

tlist = np.linspace(0, 30000, 50000)

results1 = Parallel(n_jobs=8)(delayed(spectrum_calculation)(H, tlist, wl, wm, c_ops[i], a, b) for i in tqdm(range(len(n_th))))
for key in results1[0].keys():
    df[key] = [res[key] for res in results1]

100%|██████████| 10/10 [00:00<00:00, 85.03it/s]


CPU times: total: 14.1 s
Wall time: 30min 9s


In [8]:
#REMEMBER TO CHANGE THE NAME !!!!!!!
#Name code: Omega_Hamiltonian.pkl
df.to_pickle("Results/Temperature_results/005_Full_detuned_005.pkl")

In [64]:
# With Delta=-wm, g0=0.003
df005Det=pd.read_pickle("Results/Temperature_results/005_Full_detuned_003.pkl")
df06Det=pd.read_pickle("Results/Temperature_results/06_Full_detuned_003.pkl")